In [1]:
import numpy as np
from syft.core.tensor.autodp.phi_tensor import PhiTensor
from syft.core.tensor.autodp.gamma_tensor import GammaTensor
from syft.core.tensor.fixed_precision_tensor import FixedPrecisionTensor as FPT
from syft.core.adp.data_subject_list import DataSubjectList
from syft.core.tensor.lazy_repeat_array import lazyrepeatarray as lra

/home/e/anaconda3/envs/Hagrid/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Torch Dataset equivalent

We have to implement:
- padding -> DONE!
- random horizontal flip -> DONE!
- random vertical flip -> DONE!
- random rotation -> Done but not confident about resultant LRA, or what to do with 0 values.
- normalize

In [2]:
a = PhiTensor(child=np.random.random((5,5)),data_subjects=np.ones((5,5)), min_vals=0, max_vals=1)

In [3]:
a

PhiTensor(child=FixedPrecisionTensor(child=[[25570 56399   730  8828 11760]
 [47269 58911 52585 14162 17480]
 [ 4998 36673 64755  5453 20284]
 [53661 25222 22404 18595 38348]
 [20318 40817  4667 41123 47390]]), min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 1 -> shape: (5, 5)>)

In [4]:
a.pad(width=1)

PhiTensor(child=FixedPrecisionTensor(child=[[58911 47269 58911 52585 14162 17480 14162]
 [56399 25570 56399   730  8828 11760  8828]
 [58911 47269 58911 52585 14162 17480 14162]
 [36673  4998 36673 64755  5453 20284  5453]
 [25222 53661 25222 22404 18595 38348 18595]
 [40817 20318 40817  4667 41123 47390 41123]
 [25222 53661 25222 22404 18595 38348 18595]]), min_vals=<lazyrepeatarray data: 0 -> shape: (6, 6)>, max_vals=<lazyrepeatarray data: 1 -> shape: (6, 6)>)

In [5]:
a.random_horizontal_flip()

PhiTensor(child=FixedPrecisionTensor(child=[[11760  8828   730 56399 25570]
 [17480 14162 52585 58911 47269]
 [20284  5453 64755 36673  4998]
 [38348 18595 22404 25222 53661]
 [47390 41123  4667 40817 20318]]), min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 1 -> shape: (5, 5)>)

In [6]:
a.random_vertical_flip()

PhiTensor(child=FixedPrecisionTensor(child=[[25570 56399   730  8828 11760]
 [47269 58911 52585 14162 17480]
 [ 4998 36673 64755  5453 20284]
 [53661 25222 22404 18595 38348]
 [20318 40817  4667 41123 47390]]), min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 1 -> shape: (5, 5)>)

In [7]:
a.random_rotation(degrees=20)

PhiTensor(child=FixedPrecisionTensor(child=[[    0 56547  3748     0     0]
 [    0 58095 53694 14856 17141]
 [ 6030 35822 64755  5599 19643]
 [53719 24770 22320 18059     0]
 [    0     0  4689 43159     0]]), min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 1 -> shape: (5, 5)>)

In [8]:
d = np.random.random((5,6))
d

array([[0.6934446 , 0.45189331, 0.61488947, 0.94801581, 0.51159517,
        0.11348231],
       [0.32738637, 0.65398665, 0.61949498, 0.51183404, 0.78208044,
        0.02906494],
       [0.11666689, 0.55300441, 0.38670903, 0.58121557, 0.86982795,
        0.83271507],
       [0.97768839, 0.36474543, 0.24288966, 0.35806759, 0.21061981,
        0.12508679],
       [0.78576927, 0.12810912, 0.96287653, 0.21117806, 0.88156767,
        0.42658132]])

In [11]:
from scipy.ndimage.interpolation import rotate
rotated = rotate(d, angle=20, reshape=False)

In [12]:
rotated

array([[0.        , 0.        , 0.        , 0.72643831, 0.15990978,
        0.        ],
       [0.58965845, 0.57311074, 0.55385421, 0.64559415, 0.88817223,
        0.        ],
       [0.26462835, 0.67426301, 0.44401368, 0.56402397, 0.44484788,
        0.19668674],
       [0.        , 0.51388471, 0.1776453 , 0.3437646 , 0.40383747,
        0.61286804],
       [0.        , 0.74787336, 0.34035612, 0.        , 0.        ,
        0.        ]])

In [13]:
rotated.shape

(5, 6)

<hr>
<hr>

## Model Training


To Do:
- Implement the layers needed for a ConvNet for DP Tensors:
    - Conv layer
    - BatchNorm2D
    - LeakyReLU
    - AvgPool2d
    - Linear
- Do 1 forward pass with these layers
- Do 1 backprop with these layers

In [ ]:
def conv2D(image, out_channels, kernel_size, padding=0, strides=1):
    pass

In [4]:
import torch

In [5]:
import cv2

In [7]:
img = cv2.imread("/home/e/Downloads/Dataset/10253/0/10253_idx5_x1001_y1001_class0.png")
img = cv2.resize(img, (50, 50))
img.shape

(50, 50, 3)

In [85]:
transforms.Pad(64, padding_mode="reflect")(transforms.ToPILImage()(img)).size

(178, 178)